# Criando um modelo de identificação de fraudes
- https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
- Informações da base:
    - 'Time' (tempo): contém os segundos decorridos entre cada transação e a primeira transação no conjunto de dados. 
    - 'Amount' (valor): é o valor da transação 
    - 'Class' (classe): é a variável de resposta e assume valor 1 em caso de fraude e 0 caso contrário.
    - "Infelizmente, devido a questões de confidencialidade, não podemos fornecer os recursos originais e mais informações básicas sobre os dados. Características V1, V2, … V28 são os principais componentes obtidos com PCA"

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importando o pandas
import pandas as pd

In [3]:
# Importando a base
transacoes = pd.read_csv('creditcard.csv')

<a id='modelo'></a>
### Criando um modelo de aprendizado de máquinas para identificar fraude
- Vamos criar um modelo para identificar fraude sem fazer nenhum tratamento nessa base

**Podemos ajustar as colunas Time e Amount para que elas fiquem entre 0 e 1**

In [4]:
# Primeiro para a coluna Time
transacoes.Time = transacoes.Time / transacoes.Time.max()

In [5]:
# E então para a coluna Amount
transacoes.Amount = transacoes.Amount / transacoes.Amount.max()

In [6]:
# Separando X e y
X = transacoes.drop('Class',axis=1)
y = transacoes.Class

**Separando em treino e teste**
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [7]:
# Separando em treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0,stratify=y)

### Realizando o random undersampling
- https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.RandomUnderSampler.html

In [8]:
# Importando o RandomUnderSampler do imblearn
from imblearn.under_sampling import RandomUnderSampler 

In [9]:
# Definindo o RandomUnderSampler
rus = RandomUnderSampler(random_state=42)

In [10]:
# Definindo a nova amostra
X_resRU, y_resRU = rus.fit_resample(X_train, y_train)

<a id='modelos'></a>
### Tivemos como os melhores hiperparâmetros:
- Regressão Logística:
    - {'C': 0.001, 'solver': 'liblinear'}\
- SVC:
    - {'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}
- Random Forest:
    - {'criterion': 'gini', 'max_depth': 9, 'n_estimators': 30}

In [11]:
# Importando a matriz de confusão
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score

In [12]:
# Regressão Logística
from sklearn.linear_model import LogisticRegression

clf_RL = LogisticRegression(random_state=0,
                             C=0.001,
                             solver='liblinear').fit(X_resRU, y_resRU)

y_pred_RL = clf_RL.predict(X_test)

confusion_matrix(y_test,y_pred_RL)

array([[85217,  8608],
       [   10,   152]], dtype=int64)

In [13]:
# SVC
from sklearn.svm import SVC

clf_SVC = SVC(random_state=0,
              probability=True,
             C=100,gamma='auto',
             kernel='rbf').fit(X_resRU, y_resRU)

y_pred_SVC = clf_SVC.predict(X_test)

confusion_matrix(y_test,y_pred_SVC)

array([[85791,  8034],
       [   12,   150]], dtype=int64)

In [14]:
recall_score(y_test,y_pred_SVC)

0.9259259259259259

In [15]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

clf_RF = RandomForestClassifier(max_depth=9, 
                                random_state=0,
                                criterion='gini',
                                n_estimators=30).fit(X_resRU, y_resRU)

y_pred_RF = clf_RF.predict(X_test)

confusion_matrix(y_test,y_pred_RF)

array([[91476,  2349],
       [   19,   143]], dtype=int64)

In [16]:
recall_score(y_test,y_pred_RF)

0.8827160493827161

### Podemos começar adicionando novos parâmetros

**E novamente utilizar o `GridSearchCV`**
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [17]:
# Importando o GridSearchCV
from sklearn.model_selection import GridSearchCV

In [18]:
# Adicionando novos parâmetros
parametros = {
    'C': [0.001,0.01,0.1,1,10,100,1000],
    'solver': ['newton-cg','lbfgs','liblinear','sag','saga']
}

In [19]:
# Selecionando a Regressão Logística
LogReg = LogisticRegression(random_state=42)

In [20]:
# Criando o classificador
clf_RL2 = GridSearchCV(LogReg, parametros,
                       scoring='recall'
                      ).fit(X_resRU,y_resRU)

In [21]:
# E avaliando o modelo
y_pred_RL2 = clf_RL2.predict(X_test)

confusion_matrix(y_test,y_pred_RL2)

array([[85217,  8608],
       [   10,   152]], dtype=int64)

In [22]:
# Recall apenas com C e solver
recall_score(y_test,y_pred_RL)

0.9382716049382716

In [23]:
# Recall com todos os parâmetros
recall_score(y_test,y_pred_RL2)

0.9382716049382716

### Podemos selecionar outras formas de fazer o undersampling
- Podemos utilizar o NearMiss:
    - https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.NearMiss.html#imblearn.under_sampling.NearMiss
- Ou o ClusterCentroids:
    - https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.ClusterCentroids.html

In [24]:
# Fazendo o resample
from imblearn.under_sampling import ClusterCentroids 
cc = ClusterCentroids(random_state=42)
X_resCC, y_resCC = cc.fit_resample(X_train, y_train)

**Podemos então [voltar nos modelos](#modelos) com esse novo resample**

### Além disso, também é possível adicionar novas métricas no scoring (como, por exemplo, adicionar a precisão além do recall)

In [25]:
# Criando o classificador
clf_RL3 = GridSearchCV(LogReg, parametros,
                       scoring=['recall','precision'],
                       refit='recall'
                      )

In [26]:
clf_RL3 = clf_RL3.fit(X_resRU,y_resRU)

In [27]:
# Avaliando o novo modelo
y_pred_RL3 = clf_RL3.predict(X_test)

confusion_matrix(y_test,y_pred_RL3)

array([[85217,  8608],
       [   10,   152]], dtype=int64)

### Também seria possível fazer o oversampling, lembrando que existe um custo de tempo para isso

In [28]:
# Utilizando o ADASYN para o oversampling
from imblearn.over_sampling import ADASYN 
X_resA, y_resA = ADASYN(random_state=42).fit_resample(X_train, y_train)

In [29]:
# Vamos usar o datetime só para nós dar uma ideia do tempo de processamento
import datetime as dt

In [ ]:
# Copiando exatamente o mesmo código e adicionando apenas o print da hora atual
print(dt.datetime.now())

clf_RL4 = GridSearchCV(LogReg, parametros,
                       scoring=['recall','precision'],
                       refit='recall'
                      )

clf_RL4 = clf_RL4.fit(X_resA, y_resA)

print(dt.datetime.now())

2023-11-14 00:16:04.314568
